# Segmentation test

Testing `segnet` from ['Hello AI World'](https://github.com/dusty-nv/jetson-inference#hello-ai-world).

In [ ]:
import cv2
import time
import ipywidgets
import jetson.inference
import jetson.utils
import numpy as np
import traitlets
from jetutils import GstCamera, cudaimg_to_jpeg, OUT_AS_RAW_CUDAIMG
from IPython.display import display
from sidecar import Sidecar

In [ ]:
camera = GstCamera(output_mode=OUT_AS_RAW_CUDAIMG)

Adopt segmentationBuffers from the example do
`cp jetson-inference/python/examples/segnet_utils .`

In [ ]:
from segnet_utils import segmentationBuffers
from collections import namedtuple
_ARGS = namedtuple('_ARGS','stats visualize')
tic = time.time()
net = jetson.inference.segNet('fcn-resnet18-mhp-512x320', [])
print('net initialized in {} seconds.'.format(time.time() - tic))
buffers = segmentationBuffers(net, _ARGS(True, 'overlay,mask'))

Lets use ipywidgets sidecar to display both original camera input and output from network, with some controls and debug data output.

In [ ]:
image_original = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)
image_processed = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

debug_out = ipywidgets.Textarea(value="-"*80, disabled=True,
                               layout=ipywidgets.Layout(width='1280px', height='520px'))
images_out = ipywidgets.HBox([image_original, image_processed])
alpha_slider = ipywidgets.IntSlider(value=150, min=0, max=255,
                                    description='overlay alpha',
                                    style={'description_width': 'initial'})
select_outmode = ipywidgets.ToggleButtons(options=['mask', 'overlay'],
                                     value='overlay', description='output mode',
                                     style={'description_width': 'initial'})
select_mask = ipywidgets.ToggleButton(value=True, description='mask')
select_overlay = ipywidgets.ToggleButton(value=True, description='overlay')

select_filter = ipywidgets.ToggleButtons(options=['point', 'linear'],
                                     value='point', description='filter',
                                     style={'description_width': 'initial'})
camera.read()
traitlets.dlink((camera, 'value'), (image_original, 'value'),
                transform=lambda x: cudaimg_to_jpeg(x, camera.width, camera.height))

ctrl_box = ipywidgets.HBox([ipywidgets.Label(value='output'),
                            select_mask,
                            select_overlay,
                            alpha_slider,
                            select_filter])
all_box = ipywidgets.VBox([images_out, ctrl_box, debug_out])
_sidecar = Sidecar(title='output')
with _sidecar:
    display(all_box)

Get detected segments overlay and mask. Play with controls.

In [ ]:
import IPython
ipython = IPython.get_ipython()

while True:
    ipython.kernel.do_one_iteration()
    net.SetOverlayAlpha(float(alpha_slider.value))
    # ugly way to set these directly, but..
    buffers.use_overlay = bool(select_overlay.value)
    alpha_slider.disabled = not select_overlay.value
    buffers.use_mask = bool(select_mask.value)
    buffers.use_composite = buffers.use_mask and buffers.use_overlay
    
    tic = time.time()
    frame = camera.read()
    if not buffers.use_mask and not buffers.use_overlay:
        image_processed.value = cudaimg_to_jpeg(frame, camera.width, camera.height)
        debug_out.value = 'not processing'
        continue

    buffers.Alloc(frame.shape, frame.format)
    net.Process(frame)

    if buffers.overlay:
        net.Overlay(buffers.overlay, filter_mode=select_filter.value)

    if buffers.mask:
        net.Mask(buffers.mask, filter_mode=select_filter.value)

    if buffers.composite:
        jetson.utils.cudaOverlay(buffers.overlay, buffers.composite, 0, 0)
        jetson.utils.cudaOverlay(buffers.mask, buffers.composite, buffers.overlay.width, 0)

    fps = 1.0 / (time.time() - tic)
    
    net.Mask(buffers.class_mask, buffers.grid_width, buffers.grid_height)
    class_histogram, _ = np.histogram(buffers.class_mask_np, buffers.num_classes)
    _s_fps = 'fps: {}'.format(fps)
    _s_nfps ='net fsp: {}'.format(net.GetNetworkFPS())
    _s_grid = 'grid size: {:d}x{:d}'.format(buffers.grid_width, buffers.grid_height)
    _s_ccount = 'class count: {:d}'.format(buffers.num_classes)
    _s_rows = [_s_fps, _s_nfps, _s_grid, _s_ccount,
               '-----------------------------------------',
               ' ID  class name         count          % ',
               '-----------------------------------------']
    for n in range(buffers.num_classes):
        percentage = float(class_histogram[n]) / float(buffers.grid_width * buffers.grid_height)
        _s_rows.append(' {:>2d}  {:<20s} {:>3d}   {:f}'.format(n,
                                                               net.GetClassDesc(n),
                                                               class_histogram[n],
                                                               percentage))
    debug_out.value = '\n'.join(_s_rows)

    image_processed.value = cudaimg_to_jpeg(buffers.output, camera.width, camera.height)

After interrupting kernel clean up to free gstreamer resources
if this is not done each time gstreamer ends up in situation where either reboot 
or service restart is needed.

In [ ]:
del camera
del net